In [1]:
import numpy as np
from collections import defaultdict

## 初筛

In [2]:
exp_name = 'default_3'
        
item2cate = {}
with open('../data/book14_data/book14_item_cate.txt', 'r') as f:
    for line in f:
        conts = line.strip().split(',')
        item2cate[conts[0]] = conts[1]
id2asin = {}
with open('../data/book14_data/book14_item_map.txt', 'r') as f:
    for line in f:
        conts = line.strip().split(',')
        id2asin[conts[1]] = conts[0]
id2cate = {}
with open('../data/book14_data/book14_cate_map.txt', 'r') as f:
    for line in f:
        conts = line.strip().split(',')
        id2cate[conts[-2]] = conts[-3]

In [3]:
info_path = exp_name + '_info.txt'

users = set()
user2intere = defaultdict(list)
user2cate = defaultdict(dict)
user2item = defaultdict(dict)

with open(info_path, 'r') as f:
    for line in f:
        conts = line.strip().split(',')
        user = conts[0]
        intere = conts[1]
        users.add(user)
        user2intere[user].append(intere)
        user2cate[user][intere] = set()
        user2item[user][intere] = []
        for i in range(2,52):
            item = conts[i]
            if item in item2cate:
                cate = item2cate[item]
            else:
                cate = '1'
            if cate != '1':
                user2cate[user][intere].add(cate)
                user2item[user][intere].append((id2asin[item],id2cate[cate]))

In [4]:
num = len(users)
score = dict()
for u in users:
    sco = 0
    for j in range(4):
        intere1 = user2intere[u][j]
        for k in range(j+1,4):
            intere2= user2intere[u][k]
            set1 = user2cate[u][intere1]
            set2 = user2cate[u][intere2]
            sco += len(set1&set2)/(len(set1)+len(set2))
    score[u] = sco

In [25]:
candidates = sorted(score.items(),key = lambda x:x[1],reverse = False)
i=-1

In [38]:
'''
i += 1
u = candidates[i][0]
print(i)
for k, v in user2item[u].items():
    user2item[u][k]=sorted(v,key=lambda v:v[1])
user2item[u]'''

u = '592160'
book_list = []
for k, v in user2item[u].items():
    user2item[u][k]=sorted(v,key=lambda v:v[1])
    book_list.extend([x[0] for x in v])

## 细筛

In [40]:
bookinfo = defaultdict(dict)
with open('../../meta_Books.json', 'r') as f:
    for line in f:
        r = eval(line.strip())
        asin = r['asin'] 
        if asin not in book_list:
            continue
        bookinfo[asin]['title'] = r['title']
        bookinfo[asin]['cate'] = r['category'][-1]

In [48]:
final_info = []
for k,v in user2item[u].items():
    final_info.append([(x[0], bookinfo[x[0]]['cate'], bookinfo[x[0]]['title']) for x in v])

In [54]:
import csv  
with open('./592160.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['asin', 'cate', 'title'])
    for a_list in final_info:
        for asin, cate, title in a_list:
            writer.writerow([asin, cate, title])
        writer.writerow(['*'])